In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from shapely.geometry import box, Point, Polygon
import xarray as xr
import rioxarray
from rasterio.features import shapes
import os
import folium
from math import asin, atan2, cos, degrees, radians, sin
from geocube.api.core import make_geocube

root = Path.cwd()

In [ ]:
# read in plot data
plot_data = pd.read_csv(root / 'vermont' / 'subplot_info.csv')
plot_data = plot_data.loc[~np.isnan(plot_data['fldLatitude'])]
plot_data['subplot_id'] = plot_data['PlotID'] * 10 + plot_data['SubplotID']

# convert to geodataframe
geometry = gpd.points_from_xy(plot_data['fldLongitude'], plot_data['fldLatitude'], crs='EPSG:4326')
plot_data['geometry'] = geometry
plots = gpd.GeoDataFrame(plot_data,geometry='geometry',crs='EPSG:4326')

In [ ]:
# read tree location csv
tree_info = pd.read_csv(root / 'vermont' / 'tree_info.csv')
#tree_info['subplot_id'] = tree_info['PlotID'] * 10 + tree_info['SubplotID']
tree_info['uniqueTreeID'] = (tree_info['PlotID'].astype(str) + '_' +
    tree_info['SubplotID'].astype(str) + '_' +
    tree_info['TreeID'].astype(str)
)

# read tree measurements csv
stats = pd.read_csv(root / 'vermont' / 'tree_stats.csv')
stats = stats.loc[stats['Year'].isin([2016, 2017, 2018])]
#stats['subplot_id'] = stats['PlotID'] * 10 + stats['SubplotID']
stats['uniqueTreeID'] = (stats['PlotID'].astype(str) + '_' +
    stats['SubplotID'].astype(str) + '_' +
    stats['TreeID'].astype(str)
)

# join trees to measurement info
trees = stats.merge(tree_info, on='uniqueTreeID', how='inner')
# retain columns
trees = trees[['PlotID_x','SubplotID_x','Year','Status','DBH','uniqueTreeID','Species','Azimuth','Distance']]
trees = trees.rename(columns={'DBH':'dbh_inches','PlotID_x':'PlotID','SubplotID_x':'SubplotID'})
# filter for live trees
trees = trees.loc[trees['Status']==1]

In [105]:
tree_gpd = plots.merge(trees, on=['PlotID', 'SubplotID'], how='left')

In [112]:
len(tree_gpd)

261

In [ ]:
# add taxonID column based on species codes
tree_codes = {12:'balsam_fir',97: 'spruce',261:'hemlock',315:'maple',316:'maple',318:'maple',372:'birch',531:'beech'}
tree_gpd['taxonID'] = tree_gpd['Species'].map(tree_codes)

In [ ]:
np.unique(tree_gpd['Species']) #TODO: add new species codes


array([ 12.,  97., 129., 261., 316., 318., 371., 372., 375., 403., 407.,
       409., 531., 541., 743., 762., 802., 832., 833., 837.,  nan])